# Загрузка и подготовка данных
В этом блоке происходит загрузка данных MNIST, их нормализация и преобразование меток в one-hot encoding.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка и ограничение объёма данных
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Нормализация
    y_train = tf.keras.utils.to_categorical(y_train, 10)  # One-hot encoding
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return x_train[:1000], y_train[:1000], x_test, y_test

# Загрузка данных
train_images, train_labels, test_images, test_labels = load_and_preprocess_data()

# Определение моделей
В этом блоке определяются базовая модель, модель с градиентной маскировкой и модель с регуляризацией.

In [ ]:
# Базовая модель
def build_model():
    nn_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return nn_model

# Модель с градиентной маскировкой
def build_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        tf.keras.layers.Activation('softplus')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Модель с регуляризацией и нормализацией
def build_regularized_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Реализация FGSM атаки
Генерация противоречивых примеров с использованием FGSM.

In [ ]:
# FGSM атака
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * tf.sign(gradient)
    adversarial_image = image + perturbation
    return tf.clip_by_value(adversarial_image, 0, 1)

# Генерация противоречивых примеров
def create_adversarial_samples(model, input_images, input_labels, eps):
    generated_samples = []
    for idx in range(len(input_images)):
        img = tf.convert_to_tensor(input_images[idx].reshape((1, 28, 28, 1)), dtype=tf.float32)
        lbl = tf.convert_to_tensor(input_labels[idx].reshape((1, 10)), dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(img)
            preds = model(img)
            cost = tf.keras.losses.categorical_crossentropy(lbl, preds)
        grad = tape.gradient(cost, img)
        adversarial_img = fgsm_attack(img, eps, grad)
        generated_samples.append(adversarial_img.numpy().reshape(28, 28))
    return np.array(generated_samples)

# Обучение и сохранение моделей
Этот блок содержит процесс обучения моделей и их сохранения.

In [ ]:
# Обучение базовой модели
base_model = build_model()
base_model.fit(train_images, train_labels, epochs=5)
base_model.save('base_model.h5')

# Обучение модели с градиентной маскировкой
masked_model = build_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
masked_model.save('masked_model.h5')

# Обучение модели с регуляризацией и нормализацией
regularized_model = build_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
regularized_model.save('regularized_model.h5')